# Setup

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import re
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
from tensorflow.keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas as pd

In [2]:
!conda install nomkl -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.0
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [ ]:
## Helper Functions

In [8]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

## Data Ingest

In [3]:
df= pd.read_csv('~/Documents/SOCAIL/Kaggle/train.csv')
df.comment_text = df.comment_text.str.replace('\n','').str.lower()
df['is_train'] = np.random.choice(2, size = len(df),p=[.8,.2])
df.is_train.value_counts(normalize = True)
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,is_train
0,0000997932d777bf,explanationwhy the edits made under my usernam...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,d'aww! he matches this background colour i'm s...,0,0,0,0,0,0,0
2,000113f07ec002fd,"hey man, i'm really not trying to edit war. it...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""morei can't make any real suggestions on impr...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"you, sir, are my hero. any chance you remember...",0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::and for the second time of asking, when ...",0,0,0,0,0,0,0
159567,ffea4adeee384e90,you should be ashamed of yourself that is a ho...,0,0,0,0,0,0,0
159568,ffee36eab5c267c9,"spitzer umm, theres no actual article for pros...",0,0,0,0,0,0,1
159569,fff125370e4aaaf3,and it looks like it was actually you who put ...,0,0,0,0,0,0,0


In [5]:
x_train = df[df['is_train']==1].comment_text
x_test = df[df['is_train']==0].comment_text
y_train = df[df['is_train']==1].toxic
y_test = df[df['is_train']==0].toxic

In [6]:
del df

## Load Tokenizer and Model

In [7]:
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [8]:
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

2022-01-25 16:51:13.470257: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the 

In [9]:
# Tokenize The Text
train_encodings = dbert_tokenizer(list(x_train.values),
                            truncation=True, 
                            padding=True)

test_encodings = dbert_tokenizer(list(x_test.values),
                           truncation=True, 
                           padding=True)

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),
                                    list(y_train.values)))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),
                                    list(y_test.values)))

In [11]:
BATCH_SIZE = 16
N_EPOCHS = 3

In [ ]:
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
#chose the optimizer
optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-5)
#define the loss function 
losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#build the model
dbert_model.compile(optimizer=optimizerr,
              loss=losss,
              metrics=['accuracy'])
# train the model 
dbert_model.fit(train_dataset.shuffle(len(x_train)).batch(BATCH_SIZE),
          epochs=N_EPOCHS,
          batch_size=BATCH_SIZE)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Epoch 1/3


2022-01-25 16:55:49.006800: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-25 16:55:49.025268: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


## Build Model

## Model